In [0]:
import json

bronze_params = dbutils.widgets.get("bronze_params")
print(f"Raw bronze_params: {bronze_params}")
output_data = json.loads(bronze_params)

today = output_data.get("today", "")
pays = output_data.get("pays", "")
ville = output_data.get("ville", "")
bronze_adls = output_data.get("bronze_adls", "")
silver_adls = output_data.get("silver_adls", "")
gold_adls = output_data.get("gold_adls", "")

In [0]:
df = spark.read.option("multiLine", True).json(f"{bronze_adls}/meteo/{pays}_{ville}_{today}.json")    

In [0]:
from pyspark.sql.functions import col, posexplode, to_timestamp, date_format, to_date


In [0]:
df2 = df.select("time", "temperature_2m", "soil_temperature_0cm", "precipitation", "latitude", "longitude")
# posexplode la colonne "time"
df_pos = df2.select(
    posexplode("time").alias("pos", "time"),
    col("temperature_2m"),
    col("soil_temperature_0cm"),
    col("precipitation"),
    col("latitude"),
    col("longitude")
)
# Créer le DataFrame final avec les bonnes valeurs alignées
df_final = df_pos.select(
    to_timestamp("time").alias("datetime"),
    col("temperature_2m")[col("pos")].alias("temperature_2m"),
    col("soil_temperature_0cm")[col("pos")].alias("soil_temperature_0cm"),
    col("precipitation")[col("pos")].alias("precipitation"),
    col("latitude")[col("pos")].alias("latitude"),
    col("longitude")[col("pos")].alias("longitude")
)


In [0]:
df_final = (
    df_final
    .withColumn("date", to_date("datetime"))
    .withColumn("heure", date_format("datetime", "HH:mm"))
    .drop("datetime")
)
df_final = df_final.select(
    "latitude", "longitude",
    "date",
    "heure",
    "temperature_2m",
    "soil_temperature_0cm",
    "precipitation"
)

In [0]:
df_final.show()

+--------+---------+----------+-----+--------------+--------------------+-------------+
|latitude|longitude|      date|heure|temperature_2m|soil_temperature_0cm|precipitation|
+--------+---------+----------+-----+--------------+--------------------+-------------+
|   48.85|     2.35|2025-04-13|00:00|          13.7|                14.7|          0.5|
|   48.85|     2.35|2025-04-13|01:00|          13.7|                14.5|          1.2|
|   48.85|     2.35|2025-04-13|02:00|          14.1|                14.3|          0.3|
|   48.85|     2.35|2025-04-13|03:00|          13.9|                14.2|          0.0|
|   48.85|     2.35|2025-04-13|04:00|          14.0|                14.3|          0.0|
|   48.85|     2.35|2025-04-13|05:00|          13.9|                13.9|          0.0|
|   48.85|     2.35|2025-04-13|06:00|          13.9|                14.2|          0.0|
|   48.85|     2.35|2025-04-13|07:00|          13.9|                14.2|          0.0|
|   48.85|     2.35|2025-04-13|0

In [0]:
silver_output_path = f"{silver_adls}/meteo/{pays}_{ville}_{today}.parquet"

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5248984944093824>, line 1
----> 1 silver_output_path = f"{silver_adls}/meteo"

NameError: name 'silver_adls' is not defined

In [0]:
df_final = df_final.limit(100)

In [0]:
df_final.write.mode("overwrite").parquet(silver_output_path)

In [0]:
# output_data = {
#     "today": today,
#     "silver_adls": silver_adls,
#     "gold_adls": gold_adls
# }

# output_json = json.dumps(output_data)
# dbutils.notebook.exit(output_json)